In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
from sklearn.model_selection import train_test_split  
from IPython.display import display   ## To display function results without needing a Return

from sklearn.metrics import mutual_info_score

from sklearn.feature_extraction import DictVectorizer

from sklearn.linear_model import LogisticRegression

In [4]:
!pip install wget

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wget'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9685 sha256=d047707a35132f46e5448b57770be2ff2155d168aa8b9345d86f152759a16656
  Stored in directory: /home/codespace/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [2]:
## USE WGET TO DOWNLOAD FILE

# 2. Define the URL of the CSV file you want to download
file_url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

# 3. Define the local filename to save the downloaded file
filename = 'course_lead_scoring.csv'

# 4. Use `wget` with an exclamation mark to download the file
!wget -O {filename} {file_url}

--2025-10-14 06:25:29--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.01s   

2025-10-14 06:25:29 (7.71 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [15]:
df = pd.read_csv(filename)

df.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0


# Data Preparation

In [16]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [17]:
## Replace Null values for categorical columns with 'NA'
categorical = list(df.dtypes[df.dtypes=='object'].index)

df[categorical] = df[categorical].fillna('NA')

df[categorical].isnull().sum()


lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [20]:
## Replace Null values for numerical columns with 0.0
numerical = list(df.dtypes[(df.dtypes=="int64") | (df.dtypes=="float64")].index)

df[numerical] = df[numerical].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### QUESTION 1 

What is the most frequent observation (mode) for the column industry?

In [21]:
## QUESTION 1 What is the most frequent observation (mode) for the column industry?

df.industry.mode()

0    retail
Name: industry, dtype: object

### QUESTION 2 

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [23]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [24]:
df_number = df[numerical]

df_number

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1
3,2,83843.0,1,0.87,0
4,3,85012.0,3,0.62,1
...,...,...,...,...,...
1457,1,0.0,4,0.53,1
1458,3,65259.0,2,0.24,1
1459,1,45688.0,3,0.02,1
1460,5,71016.0,0,0.25,1


In [25]:
df_number.corr().abs()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,0.023565,0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


What are the two features that have the biggest correlation?

interaction_count and lead_score  --> 0.0009888

number_of_courses_viewed and lead_score --> 0.004879

number_of_courses_viewed and interaction_count  --> 0.023565

annual_income and interaction_count  --> 0.027036

Only consider the pairs above when answering this question.  

ANSWER = The 2 highest correlations are shared between {"number_of_courses_viewed" and "interaction_count"} 

and {"annual_income" and "interaction_count"}



# QUESTION 3 Split Data into Train/Validation/Test datasets


Split the data

Split your data in train/val/test sets with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value converted is not in your dataframe.

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
## set 80/20  Train/Test split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## Split Traiining set into training and validation
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [28]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [29]:
y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [30]:
df_val.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,healthcare,3,52220.0,unemployed,europe,1,0.07
1,organic_search,technology,3,59656.0,unemployed,middle_east,4,0.65


### QUESTION 3

In [31]:
from sklearn.metrics import mutual_info_score

In [96]:
### Calculate mutual information score between 'converted' and other categorical variables

df_full_train.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1


In [32]:
## sklearn mutual info score takes a series to series comparison as its paramters
    ## so create function and .apply() the function to each column individually

def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [33]:
### apply mutual_info_score method to each filtered categorical column using defined function and .apply() 
    ## save results into new DataFrame and sort results

mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

### QUESTION 3 ANSWER = "lead_source" variable has the largest mutual information score with 'y' variable 'converted'

# QUESTION 4

In [34]:
## ONE HOT ENCODING USING SKLEARN 
    ## NOTE THAT THE VECTORIZER TAKES DICTIONARY AS ITS INPUT

from sklearn.feature_extraction import DictVectorizer

In [36]:
df_train.head(2)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98
...,...,...,...,...,...,...,...,...
871,organic_search,other,1,43907.0,employed,australia,4,0.33
872,social_media,retail,3,64969.0,employed,north_america,1,0.18
873,NA,education,3,89042.0,employed,asia,4,0.75
874,social_media,manufacturing,1,0.0,self_employed,europe,1,0.65


In [39]:
#must remove 'converted' from list of numerical columns
    ## use list comprehension
numerical = [string for string in numerical if string != "converted"]
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [40]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [41]:
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))